In [5]:
import pandas as pd
import openpyxl
import csv
import requests
from bs4 import BeautifulSoup
from celery import Celery
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException 
import re
import time
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import geocoder
from tqdm import tqdm
import random
import glob

## 01 批次抓取建照查詢的回傳連結

1.製作3個年度的清單與編號
2.放入查詢網站查詢
3.(取消)製作爬取時的時間標記

#### 製作109-112年度的清單與編號

In [19]:
# 建照發照數量可從中華民國統計資訊網查到(https://www.stat.gov.tw/Statistics.aspx?n=3020&CaN=494)
# 109年
years_109 = [109] * 263
numbers_109 = ['{:04d}'.format(i) for i in range(1,264)]
df_109 = pd.DataFrame({"年份":years_109, "號碼":numbers_109})

# 110年
years_110 = [110] * 337
numbers_110 = ['{:04d}'.format(i) for i in range(1,338)]
df_110 = pd.DataFrame({"年份":years_110, "號碼":numbers_110})

# 111年
years_111 = [111] * 324
numbers_111 = ['{:04d}'.format(i) for i in range(1,325)]
df_111 = pd.DataFrame({"年份":years_111, "號碼":numbers_111})

# 112年
years_112 = [112] * 226
numbers_112 = ['{:04d}'.format(i) for i in range(1,227)]
df_112 = pd.DataFrame({"年份":years_112, "號碼":numbers_112})

# 合併以上三個年度
df_all = pd.concat([df_109, df_110, df_111, df_112], ignore_index=True)
df_all["存在"] = "N" 

# 因爬蟲中斷而取未完成部分
df_all_new = df_all.loc[852:]
print(df_all_new)

       年份    號碼 存在
852   111  0253  N
853   111  0254  N
854   111  0255  N
855   111  0256  N
856   111  0257  N
...   ...   ... ..
1145  112  0222  N
1146  112  0223  N
1147  112  0224  N
1148  112  0225  N
1149  112  0226  N

[298 rows x 3 columns]


#### 製作106-108年度的清單與編號

In [2]:
# 建照發照數量可從中華民國統計資訊網查到(https://www.stat.gov.tw/Statistics.aspx?n=3020&CaN=494)
# 106年
years_106 = [106] * 205
numbers_106 = ['{:04d}'.format(i) for i in range(1,206)]
df_106 = pd.DataFrame({"年份":years_106, "號碼":numbers_106})

# 107年
years_107 = [107] * 223
numbers_107 = ['{:04d}'.format(i) for i in range(1,224)]
df_107 = pd.DataFrame({"年份":years_107, "號碼":numbers_107})

# 108年
years_108 = [108] * 263
numbers_108 = ['{:04d}'.format(i) for i in range(1,264)]
df_108 = pd.DataFrame({"年份":years_108, "號碼":numbers_108})

# 合併以上三個年度
df_all = pd.concat([df_106, df_107, df_108], ignore_index=True)
df_all["存在"] = "N" 

# print(df_all)

# 因爬蟲中斷而取未完成部分
df_all_new = df_all.loc[569:]
print(df_all_new)

      年份    號碼 存在
569  108  0142  N
570  108  0143  N
571  108  0144  N
572  108  0145  N
573  108  0146  N
..   ...   ... ..
686  108  0259  N
687  108  0260  N
688  108  0261  N
689  108  0262  N
690  108  0263  N

[122 rows x 3 columns]


#### 爬蟲部分

In [4]:
df_110.columns

Index(['年份', '號碼', '存在'], dtype='object')

In [125]:
# 目標網站的 URL
url = "https://tccmoapply.dba.tcg.gov.tw/tccmoapply/maliapp/asp/resinf01_m000servlet.do?node=20190322174610910001"
chrome_path = r"C:\Users\doit\anaconda3\envs\VSenv\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_path)
driver.get(url)

In [126]:
# 找到查詢建照的button
button_entry = driver.find_element_by_id("20190322174610910001")
button_entry.click()

In [3]:
crawler_save = pd.DataFrame()

# 以x = 0計數
# x = 0

# 獲取地址總數
total_cases = len(df_all_new)

# 使用tqdm顯示進度
tqdm.pandas(desc="Scrawling Building Cases", total=total_cases)

for index, row in tqdm(df_all_new.iterrows(), total=total_cases, desc="Scrawling Building Cases"):
    url = "https://tccmoapply.dba.tcg.gov.tw/tccmoapply/maliapp/asp/resinf01_m000servlet.do?node=20190322174610910001"
    chrome_path = r"C:\Users\doit\anaconda3\envs\VSenv\chromedriver-win64\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chrome_path)
    driver.get(url)
    delay = random.uniform(1,3)
    time.sleep(delay)

    # 找到查詢建照的button
    button_entry = driver.find_element_by_id("20190322174610910001")
    button_entry.click()

    # 找到年份搜尋欄位
    element_year = driver.find_element_by_id("Qry_LICENSE_YY")
    element_year.send_keys(str(row["年份"]))

    # 找到建照號碼搜尋欄位
    element_number = driver.find_element_by_id("Qry_LICENSE_NO1")
    element_number.send_keys(str(row["號碼"]))

    # 點擊搜尋
    button_search = driver.find_element_by_id("QueryParamButton_executeQuery")
    button_search.click()

    time.sleep(2)
    try:
        # 點擊建照號碼
        link_xpath = '//a[starts-with(@onclick, "run_view(")]'
        driver.find_element(By.XPATH, link_xpath).click()
        
        time.sleep(2)
        #  定位建築地點的元素
        location_element_01 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "建照號碼")]')
        location_element_02 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "使照號碼")]')
        location_element_03 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "開工日期")]')
        location_element_04 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "竣工日期")]')
        location_element_05 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "建築規模")]')
        location_element_06 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "戶數")]')
        location_element_07 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "建築地點:地號")]')
        location_element_08 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "建築地點:地址")]')
        location_element_09 = driver.find_element_by_xpath('//td[@class="name text-right" and contains(text(), "姓名")]')

        # 獲取建築地點的值
        location_value_01 = location_element_01.find_element_by_xpath('./following-sibling::td').text
        location_value_02 = location_element_02.find_element_by_xpath('./following-sibling::td').text
        location_value_03 = location_element_03.find_element_by_xpath('./following-sibling::td').text
        location_value_04 = location_element_04.find_element_by_xpath('./following-sibling::td').text
        location_value_05 = location_element_05.find_element_by_xpath('./following-sibling::td').text
        location_value_06 = location_element_06.find_element_by_xpath('./following-sibling::td').text
        location_value_07 = location_element_07.find_element_by_xpath('./following-sibling::td').text
        location_value_08 = location_element_08.find_element_by_xpath('./following-sibling::td').text
        location_value_09 = location_element_09.find_element_by_xpath('./following-sibling::td').text

        # 打印 DataFrame
        data = [{
            "建照號碼": location_value_01,
            "使照號碼": location_value_02,
            "開工日期": location_value_03,
            "竣工日期": location_value_04,
            "建築規模": location_value_05,
            "戶數": location_value_06,
            "建築地點:地號": location_value_07,
            "建築地點:地址": location_value_08,
            "姓名": location_value_09
        }]
        data_df = pd.DataFrame(data)
        crawler_save = pd.concat([crawler_save, data_df], ignore_index=True)
    except NoSuchElementException:
        # 如果找不到連結，印出訊息並繼續下一個迴圈
        print(f"找不到建照號碼為 {location_value_01} 的連結")
        data = [{
            "建照號碼": None,
            "使照號碼": None,
            "開工日期": None,
            "竣工日期": None,
            "建築規模": None,
            "戶數": None,
            "建築地點:地號": None,
            "建築地點:地址": None,
            "姓名": None
        }]
        crawler_save = pd.concat([crawler_save, data_df], ignore_index=True)
        driver.quit()
        continue
    # x += 1
    # print("已完成爬取{}/1150筆資料".format(x))
    # 關閉瀏覽器
    driver.quit()
    

crawler_save.tail()

Scrawling Building Cases:   0%|          | 0/122 [00:00<?, ?it/s]

Scrawling Building Cases:   2%|▏         | 2/122 [00:30<29:44, 14.87s/it]

找不到建照號碼為 108建0143號 的連結


Scrawling Building Cases:  10%|▉         | 12/122 [02:25<21:18, 11.62s/it]

找不到建照號碼為 108建0153號 的連結


...

找不到建照號碼為 108建0259號 的連結


Scrawling Building Cases:  99%|█████████▉| 121/122 [23:15<00:10, 10.10s/it]

找不到建照號碼為 108建0259號 的連結


Scrawling Building Cases: 100%|██████████| 122/122 [23:25<00:00, 11.52s/it]


,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
117,108建0259號,112使0100號,109/03/06,112/04/24,地上14層，地下3層,17層 49戶,大同區市府段二小段0367-0000等6筆,103大同區103臺北市大同區建泰里長安西路,三豐建設股份有限公司 負責人：洪敏夫
118,108建0259號,112使0100號,109/03/06,112/04/24,地上14層，地下3層,17層 49戶,大同區市府段二小段0367-0000等6筆,103大同區103臺北市大同區建泰里長安西路,三豐建設股份有限公司 負責人：洪敏夫
119,108建0259號,112使0100號,109/03/06,112/04/24,地上14層，地下3層,17層 49戶,大同區市府段二小段0367-0000等6筆,103大同區103臺北市大同區建泰里長安西路,三豐建設股份有限公司 負責人：洪敏夫
120,108建0259號,112使0100號,109/03/06,112/04/24,地上14層，地下3層,17層 49戶,大同區市府段二小段0367-0000等6筆,103大同區103臺北市大同區建泰里長安西路,三豐建設股份有限公司 負責人：洪敏夫
121,108建0259號,112使0100號,109/03/06,112/04/24,地上14層，地下3層,17層 49戶,大同區市府段二小段0367-0000等6筆,103大同區103臺北市大同區建泰里長安西路,三豐建設股份有限公司 負責人：洪敏夫


In [47]:
crawler_save.tail()
# print(crawler_save.shape)

,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
99,108建0137號,109使0031號,,108/11/26,地上2層，地下0層,2層 1戶,大同區圓環段三小段0560-0022等4筆,103大同區103臺北市大同區星明里寧夏路27號,財團法人陳德星堂 陳春銅
100,108建0138號,112使0016號,109/07/07,111/02/21,地上4層，地下2層,6層 4戶,內湖區康寧段一小段0424-0000等1筆,114內湖區114臺北市內湖區秀湖里大湖街166巷44號,高楊碧菁
101,108建0139號,111使0051號,109/05/04,110/10/19,地上2層，地下0層,2層 1戶,內湖區西湖段二小段0037-0000等1筆,114內湖區114臺北市內湖區西安里內湖路一段91巷229號（旁）,許金蘭
102,108建0140號,,109/05/12,,地上24層，地下4層,28層 290戶,松山區寶清段一小段0057-0013等1筆,105松山區105臺北市松山區自強里延壽街,中華工程股份有限公司 代表人：朱蕙蘭
103,108建0141號,112使0013號,109/05/07,111/11/14,地上15層，地下2層,17層 102戶,中山區德惠段二小段0813-0001等6筆,104中山區104臺北市中山區新福里民權東路二段,昇陽建設企業股份有限公司 負責人：麥寬成


In [4]:
crawler_save.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Crawled\crawler_save_106-108_07.xlsx")

## 02 處理建照資料

#### 導入與去除重複值 (僅讀入2個檔案)

In [110]:
# 讀入二個表單
list_01 = pd.read_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Crawled\crawler_save_01_1215.xlsx")
list_02 = pd.read_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Crawled\crawler_save_02_1215.xlsx")

# 合併表單
raw_data = pd.concat([list_01, list_02], ignore_index=True)
raw_data.head()

# 去除重複列
data = raw_data.drop_duplicates(subset=["建照號碼"], keep="first")

# 重設索引
data = data.reset_index(drop=True)

data.shape #(963, 10)
data.head()

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
0,0,109建0001號,NaN,109/10/05,NaN,地上21層，地下6層,27層 77戶,中正區河堤段一小段0216-0000等26筆,100中正區100臺北市中正區頂東里里羅斯福路三段68之1號,北城建設開發股份有限公司 負責人：洪春木
1,1,109建0002號,NaN,109/10/05,NaN,地上12層，地下3層,15層 122戶,南港區經貿段0065-0000等1筆,115南港區115臺北市南港區南港里惠民街,臺北市政府都市發展局 局長:王玉芬
2,2,109建0003號,NaN,109/03/23,NaN,地上12層，地下4層,16層 33戶,信義區永吉段四小段0105-0000等7筆,110信義區110臺北市信義區富台里忠孝東路五段423巷,僑馥建築經理股份有限公司 負責人：彭慶
3,4,109建0005號,NaN,109/12/12,NaN,地上5層，地下1層,6層 1戶,中山區正義段一小段0472-0000等2筆,104中山區104臺北市中山區正守里林森北路67巷,嘉春建設股份有限公司 負責人:江顯方
4,5,109建0006號,NaN,109/10/20,NaN,地上10層，地下3層,13層 29戶,中正區臨沂段一小段0314-0000等3筆,100中正區100臺北市中正區幸市里臨沂街25巷1號,僑馥建築經理股份有限公司 負責人:彭慶


#### 導入與去除重複值 (批次讀入多個檔案)

In [49]:
# 指定檔案路徑及檔案名稱開頭
file_path = r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Crawled"
file_prefix = "crawler_save"

# 取得符合條件的所有檔案
files = glob.glob(file_path + '\\' + file_prefix + '*.xlsx')

# 初始化一個空的 DataFrame 來存放合併後的資料
merged_data = pd.DataFrame()

# 逐一讀取並合併檔案
for file in files:
    df = pd.read_excel(file)
    merged_data = pd.concat([merged_data, df], ignore_index=True)

merged_data.shape #(1846, 10)

(1846, 10)

In [14]:
merged_data.head(2)

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
0,0,109建0001號,NaN,109/10/05,NaN,地上21層，地下6層,27層 77戶,中正區河堤段一小段0216-0000等26筆,100中正區100臺北市中正區頂東里里羅斯福路三段68之1號,北城建設開發股份有限公司 負責人：洪春木
1,1,109建0002號,NaN,109/10/05,NaN,地上12層，地下3層,15層 122戶,南港區經貿段0065-0000等1筆,115南港區115臺北市南港區南港里惠民街,臺北市政府都市發展局 局長:王玉芬


In [50]:
# 確認缺失值(開工日期：133)
merged_data.isna().sum()


Unnamed: 0       0
建照號碼             0
使照號碼          1282
開工日期           133
竣工日期          1293
建築規模             0
戶數               0
建築地點:地號         39
建築地點:地址          0
姓名              19
dtype: int64

In [51]:
# 確認缺少開工日期的欄位
missing_dates = merged_data[merged_data["開工日期"].isna()].index.tolist()

# 顯示缺失欄位
filtered_date_na = merged_data.loc[missing_dates]
filtered_date_na.head(2)

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
58,58,109建0059號,NaN,NaN,NaN,地上5層，地下1層,6層 1戶,中山區長安段一小段0421-0000等1筆,104中山區104臺北市中山區力行里遼寧街45巷24號2樓,陳勤政
89,89,109建0090號,NaN,NaN,NaN,地上8層，地下2層,10層 49戶,北投區崇仰段三小段0183-0000等14筆,112北投區112臺北市北投區永欣里石牌路二段315巷16弄（2號對面空地）,龍霖開發建設股份有限公司 負責人:李建霖


In [52]:
# 去除缺少開工日期的建案
# 保留原始的 DataFrame 並刪除缺失值的行
merged_data = merged_data.dropna(subset=["開工日期"])
merged_data.shape #(1713, 10)
merged_data.head(2)

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
0,0,109建0001號,NaN,109/10/05,NaN,地上21層，地下6層,27層 77戶,中正區河堤段一小段0216-0000等26筆,100中正區100臺北市中正區頂東里里羅斯福路三段68之1號,北城建設開發股份有限公司 負責人：洪春木
1,1,109建0002號,NaN,109/10/05,NaN,地上12層，地下3層,15層 122戶,南港區經貿段0065-0000等1筆,115南港區115臺北市南港區南港里惠民街,臺北市政府都市發展局 局長:王玉芬


#### 開工日期轉換為西元年(針對欄與列處理)

In [47]:
# 以單一欄為目標對象
def column_transfer_to_time(column):
    # 拆分民國年與時間 
    date = column.astype(str)
    
    # 添加條件檢查，確保 roc_year 不是空字符串
    roc_year = date.str.split('/').str[0]
    mask = (roc_year != '') & (~roc_year.isna())  # 檢查不是空字符串且不是 NaN
    roc_year = roc_year[mask]

    if not roc_year.empty:
        # 民國轉西元
        month = date.str.split('/').str[1]
        day = date.str.split('/').str[2]
        
        # 在進行 astype(int) 之前，檢查是否有 NaN
        if not roc_year.str.contains('nan').any():
            ad_year = roc_year.astype(int) + 1911
            ad_date = ad_year.astype(str).str.cat([month, day], sep='/')

            # 轉換為時間格式
            new_date = pd.to_datetime(ad_date, format='%Y/%m/%d')
            column = new_date
    else:
        return column  # 如果 roc_year 為空，直接返回原始 column，避免資料遺失

    return column  # 將修改後的 column 返回

# 以單一列為目標對象
def column_transfer_to_time_row(row):
    # 拆分民國年與時間 
    date = str(row)
    
    # 添加條件檢查，確保 roc_year 不是空字符串
    date_parts = date.split('/')
    
    if len(date_parts) >= 3:
        roc_year = date_parts[0]
        month = date_parts[1]
        day = date_parts[2]
        
        # 在進行 astype(int) 之前，檢查是否有 NaN
        if 'nan' not in roc_year:
            ad_year = int(roc_year) + 1911
            ad_date = f"{ad_year}/{month}/{day}"

            # 轉換為時間格式
            new_date = pd.to_datetime(ad_date, format='%Y/%m/%d')
            return new_date
    return row


In [53]:
# 開工日期轉換
merged_data["開工日期"] = column_transfer_to_time(merged_data["開工日期"])
merged_data["竣工日期"] = merged_data["竣工日期"].apply(column_transfer_to_time_row)
merged_data.head()

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
0,0,109建0001號,NaN,2020-10-05,NaT,地上21層，地下6層,27層 77戶,中正區河堤段一小段0216-0000等26筆,100中正區100臺北市中正區頂東里里羅斯福路三段68之1號,北城建設開發股份有限公司 負責人：洪春木
1,1,109建0002號,NaN,2020-10-05,NaT,地上12層，地下3層,15層 122戶,南港區經貿段0065-0000等1筆,115南港區115臺北市南港區南港里惠民街,臺北市政府都市發展局 局長:王玉芬
2,2,109建0003號,NaN,2020-03-23,NaT,地上12層，地下4層,16層 33戶,信義區永吉段四小段0105-0000等7筆,110信義區110臺北市信義區富台里忠孝東路五段423巷,僑馥建築經理股份有限公司 負責人：彭慶
3,3,109建0003號,NaN,2020-03-23,NaT,地上12層，地下4層,16層 33戶,信義區永吉段四小段0105-0000等7筆,110信義區110臺北市信義區富台里忠孝東路五段423巷,僑馥建築經理股份有限公司 負責人：彭慶
4,4,109建0005號,NaN,2020-12-12,NaT,地上5層，地下1層,6層 1戶,中山區正義段一小段0472-0000等2筆,104中山區104臺北市中山區正守里林森北路67巷,嘉春建設股份有限公司 負責人:江顯方


#### 地址正規化與增加經緯度

In [54]:
# 由於地址格式固定，故直接以字串長度取後方地址
merged_data["建築地點:地址"] = merged_data["建築地點:地址"].str[9:]
merged_data.head()

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
0,0,109建0001號,NaN,2020-10-05,NaT,地上21層，地下6層,27層 77戶,中正區河堤段一小段0216-0000等26筆,臺北市中正區頂東里里羅斯福路三段68之1號,北城建設開發股份有限公司 負責人：洪春木
1,1,109建0002號,NaN,2020-10-05,NaT,地上12層，地下3層,15層 122戶,南港區經貿段0065-0000等1筆,臺北市南港區南港里惠民街,臺北市政府都市發展局 局長:王玉芬
2,2,109建0003號,NaN,2020-03-23,NaT,地上12層，地下4層,16層 33戶,信義區永吉段四小段0105-0000等7筆,臺北市信義區富台里忠孝東路五段423巷,僑馥建築經理股份有限公司 負責人：彭慶
3,3,109建0003號,NaN,2020-03-23,NaT,地上12層，地下4層,16層 33戶,信義區永吉段四小段0105-0000等7筆,臺北市信義區富台里忠孝東路五段423巷,僑馥建築經理股份有限公司 負責人：彭慶
4,4,109建0005號,NaN,2020-12-12,NaT,地上5層，地下1層,6層 1戶,中山區正義段一小段0472-0000等2筆,臺北市中山區正守里林森北路67巷,嘉春建設股份有限公司 負責人:江顯方


In [55]:
# 重新依照日期排序
merged_data = merged_data.sort_values("建照號碼")
merged_data.head()

,Unnamed: 0,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名
1150,0,106建0001號,109使0023號,2017-10-03,2019-09-19,地上15層，地下3層,18層 52戶,萬華區龍山段一小段0698-0000等7筆,臺北市萬華區青山里環河南路2段,中華建築經理股份有限公司 負責人:鍾榮吉
1151,1,106建0001號,109使0023號,2017-10-03,2019-09-19,地上15層，地下3層,18層 52戶,萬華區龍山段一小段0698-0000等7筆,臺北市萬華區青山里環河南路2段,中華建築經理股份有限公司 負責人:鍾榮吉
1152,2,106建0003號,109使0075號,2017-10-04,2020-01-06,地上12層，地下3層,15層 73戶,中山區中山段三小段0332-0000等9筆,臺北市中山區中山里中山北路二段65巷,僑馥建築經理股份有限公司
1153,3,106建0003號,109使0075號,2017-10-04,2020-01-06,地上12層，地下3層,15層 73戶,中山區中山段三小段0332-0000等9筆,臺北市中山區中山里中山北路二段65巷,僑馥建築經理股份有限公司
1154,4,106建0005號,NaN,2017-10-09,NaT,地上21層，地下3層,24層 139戶,文山區萬芳段二小段0182-0001等33筆,臺北市文山區萬芳里萬芳路,潤隆建設股份有限公司 負責人:蔡聰賓


In [56]:
merged_data.isna().sum()

Unnamed: 0       0
建照號碼             0
使照號碼          1170
開工日期             0
竣工日期          1170
建築規模             0
戶數               0
建築地點:地號         39
建築地點:地址          0
姓名              19
dtype: int64

In [57]:
# geocoder 功能
def get_lat_long(address):
    location = geocoder.arcgis(address)
    
    if location is not None and location.ok:
        lat, long = location.latlng
        return lat, long
    else:
        # 在無法找到經緯度時，返回一個預設值或引發異常
        return None, None

In [62]:
# 獲取地址總數
total_addresses = len(merged_data["建築地點:地址"])

# 使用tqdm顯示進度
tqdm.pandas(desc="Adding WGS84 Coordinates", total=total_addresses)

# 新增WGS84經緯度
merged_data["lat_WGS84"], merged_data["lng_WGS84"] = zip(*merged_data["建築地點:地址"].progress_apply(get_lat_long))


Adding WGS84 Coordinates: 100%|██████████| 1713/1713 [38:44<00:00,  1.36s/it] 


In [63]:
merged_data.isna().sum()

Unnamed: 0       0
建照號碼             0
使照號碼          1170
開工日期             0
竣工日期          1170
建築規模             0
戶數               0
建築地點:地號         39
建築地點:地址          0
姓名              19
lat_WGS84       37
lng_WGS84       37
dtype: int64

In [67]:
merged_data.head(2)

,建照號碼,使照號碼,開工日期,竣工日期,建築規模,戶數,建築地點:地號,建築地點:地址,姓名,lat_WGS84,lng_WGS84
1150,106建0001號,109使0023號,2017-10-03,2019-09-19,地上15層，地下3層,18層 52戶,萬華區龍山段一小段0698-0000等7筆,臺北市萬華區青山里環河南路2段,中華建築經理股份有限公司 負責人:鍾榮吉,NaN,NaN
1151,106建0001號,109使0023號,2017-10-03,2019-09-19,地上15層，地下3層,18層 52戶,萬華區龍山段一小段0698-0000等7筆,臺北市萬華區青山里環河南路2段,中華建築經理股份有限公司 負責人:鍾榮吉,25.037655,121.496415


In [66]:
# 去除不需要的unnamed欄位
merged_data = merged_data.drop(columns="Unnamed: 0")
merged_data.head()

KeyError: "['Unnamed: 0'] not found in axis"

In [69]:
# 儲存成excel與CSV
merged_data.to_csv(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\106-112建照.csv")
merged_data.to_excel(r"D:\Chu's Document!\02 Project\06 道路塌陷防治專案(天坑)\03 Data\Processed\106-112建照.xlsx")
print("done")

done
